# Fully Convolutional Networks (FCNs)
![](https://drive.google.com/uc?id=1jaU0zy7djT0W698uLp80OMNWkEYSa_CR)

In [1]:
import torch
import torch.nn.functional as F

In [16]:
import torchvision.models as models
vgg = models.vgg16(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# pool3
# pool4
# final-output

In [4]:
from torchsummary import summary

In [5]:
summary(vgg, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [6]:
import torch.nn as nn

In [17]:
class FCN8(nn.Module):
  def __init__(self, num_classes=2):
    super(FCN8, self).__init__()
    vgg = models.vgg16(pretrained=True)
    self.features = vgg.features # till convolution and pooling layers

    # extract intermediate feature maps
    self.pool3 = self.features[:17]
    self.pool4 = self.features[:24]
    self.pool5 = self.features

    # 1x1 convolutions to convert to class scores
    self.conv1x1_3 = nn.Conv2d(256, num_classes, kernel_size=1)
    self.conv1x1_4 = nn.Conv2d(512,num_classes, kernel_size=1)
    self.conv1x1_5 = nn.Conv2d(512, num_classes, kernel_size=1)

    self.upsample32 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, stride=2, padding=1)
    self.upsample16 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, stride=2, padding=1)
    self.upsample8 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=16, stride=8, padding=4)

  def forward(self, x):
    pool3_out = self.pool3(x)
    pool4_out = self.pool4(x)
    pool5_out = self.pool5(x)

    pool3_out = self.conv1x1_3(pool3_out)
    pool4_out = self.conv1x1_4(pool4_out)
    pool5_out = self.conv1x1_5(pool5_out)

    x = self.upsample32(pool5_out) + pool4_out
    x = self.upsample16(x) + pool3_out
    x = self.upsample8(x)

    return x


In [ ]:
# I = "input size"
# S = "Stride"
# K = "Kernel Size"
# P = "Padding"
# O = (I - 1) * S + K - 2*P
# reverse of previous formula
# input_size = 224, 224
# max_pool5 = 7, 7 == 32x
# max_pool4 = 14, 14 == 16x
# max_pool3 = 28, 28 == 8x

In [9]:
224/7

32.0

In [19]:
# dummy input
model = FCN8(num_classes=1000)
dummy_input = torch.randn(1,3, 224, 224)
output = model(dummy_input)
print("Output shape : ", output.shape)

Output shape :  torch.Size([1, 1000, 224, 224])
